In [1]:
print("hello11")

hello11


In [2]:
!pip install librosa numpy pandas torch tqdm scikit-learn

  Using cached librosa-0.10.2.post1-py3-none-any.whl.metadata (8.6 kB)
  Using cached torch-2.3.1-cp312-cp312-win_amd64.whl.metadata (26 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached soundfile-0.12.1-py2.py3-none-win_amd64.whl.metadata (14 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached soxr-0.3.7-cp312-cp312-win_amd64.whl.metadata (5.7 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
  Using cached tbb-2021.13.0-py3-none-win_amd64.whl.metadata (1.1 kB)
Using cached librosa-0.10.2.post1-py3-none-any.whl (260 kB)
Using cached torch-2.3.1-cp312-cp312-win_amd64.whl (159.7 MB)
Using cached audioread-3.0.1-py3-none-any.whl (23 kB)
   ---------------------------------------- 0.0/228.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/228.5 MB 640.0 kB/s eta 0:05:57
   -------------------------------

In [4]:
import os
print(os.getcwd())


C:\Users\82102


In [6]:
file_path = r'C:\Users\82102\Desktop\sw-데이콘-음성AI\train.csv'
df = pd.read_csv(file_path)
print(df.head())


         id                  path label
0  RUNQPNJF  ./train/RUNQPNJF.ogg  real
1  JFAWUOGJ  ./train/JFAWUOGJ.ogg  fake
2  RDKEKEVX  ./train/RDKEKEVX.ogg  real
3  QYHJDOFK  ./train/QYHJDOFK.ogg  real
4  RSPQNHAO  ./train/RSPQNHAO.ogg  real


In [9]:
os.chdir('C:/Users/82102/Desktop/sw-데이콘-음성AI')
print(os.getcwd())

C:\Users\82102\Desktop\sw-데이콘-음성AI


In [10]:
import os
print(os.getcwd())

C:\Users\82102\Desktop\sw-데이콘-음성AI


In [1]:
import librosa
import numpy as np
import pandas as pd
import random
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import warnings
import os

warnings.filterwarnings('ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class Config:
    SR = 32000
    N_MFCC = 13
    ROOT_FOLDER = './'
    N_CLASSES = 2
    BATCH_SIZE = 96
    N_EPOCHS = 100
    LR = 3e-4
    SEED = 42

CONFIG = Config()

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CONFIG.SEED)

df = pd.read_csv('train.csv')
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CONFIG.SEED)

def get_mfcc_feature(df, train_mode=True):
    features = []
    labels = []
    for _, row in tqdm(df.iterrows()):
        y, sr = librosa.load(row['path'], sr=CONFIG.SR)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CONFIG.N_MFCC)
        mfcc = np.mean(mfcc.T, axis=0)
        features.append(mfcc)
        if train_mode:
            label = row['label']
            label_vector = np.zeros(CONFIG.N_CLASSES, dtype=float)
            label_vector[0 if label == 'fake' else 1] = 1
            labels.append(label_vector)
    if train_mode:
        return features, labels
    return features

train_mfcc, train_labels = get_mfcc_feature(train, True)
val_mfcc, val_labels = get_mfcc_feature(val, True)

class CustomDataset(Dataset):
    def __init__(self, mfcc, label):
        self.mfcc = mfcc
        self.label = label
    def __len__(self):
        return len(self.mfcc)
    def __getitem__(self, index):
        if self.label is not None:
            return self.mfcc[index], self.label[index]
        return self.mfcc[index]

train_dataset = CustomDataset(train_mfcc, train_labels)
val_dataset = CustomDataset(val_mfcc, val_labels)

train_loader = DataLoader(train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)

class MLP(nn.Module):
    def __init__(self, input_dim=CONFIG.N_MFCC, hidden_dim=128, output_dim=CONFIG.N_CLASSES):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = torch.sigmoid(x)
        return x

def mixup_data(x, y, alpha=1.0):
    '''MixUp 데이터 증강 기법 적용'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    '''MixUp 손실 계산'''
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

def sharpen(p, T=0.5):
    '''예측 값을 균일화 (Sharpening)'''
    p = p ** (1 / T)
    return p / p.sum(dim=1, keepdim=True)

def mixmatch(model, x_l, y_l, x_ul, alpha=0.75, T=0.5, K=2):
    '''MixMatch 알고리즘 적용'''
    # Unlabeled 데이터에 대한 K개의 예측 평균 및 Sharpening 적용
    with torch.no_grad():
        logits = torch.zeros(K, x_ul.size(0), CONFIG.N_CLASSES).to(device)
        for k in range(K):
            logits[k] = model(x_ul)
        q_bar = torch.mean(F.softmax(logits, dim=2), dim=0)
        q = sharpen(q_bar, T)

    # Labeled 데이터와 Unlabeled 데이터를 혼합
    x = torch.cat([x_l, x_ul], dim=0)
    y = torch.cat([y_l, q], dim=0)
    mixed_x, y_a, y_b, lam = mixup_data(x, y, alpha)

    # Labeled 데이터와 Unlabeled 데이터로 나누기
    mixed_x_l = mixed_x[:x_l.size(0)]
    mixed_x_ul = mixed_x[x_l.size(0):]
    y_l_a, y_l_b = y_a[:x_l.size(0)], y_b[:x_l.size(0)]
    y_ul_a, y_ul_b = y_a[x_l.size(0):], y_b[x_l.size(0):]

    return mixed_x_l, y_l_a, y_l_b, mixed_x_ul, y_ul_a, y_ul_b, lam

def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    best_val_score = 0
    best_model = None

    for epoch in range(1, CONFIG.N_EPOCHS + 1):
        model.train()
        train_loss = []
        for batch in tqdm(train_loader):
            features, labels = batch
            features = features.float().to(device)
            labels = labels.float().to(device)

            # MixMatch 적용
            mixed_x_l, y_l_a, y_l_b, mixed_x_ul, y_ul_a, y_ul_b, lam = mixmatch(
                model, features, labels, features, alpha=0.75, T=0.5, K=2)

            optimizer.zero_grad()
            output_l = model(mixed_x_l)
            output_ul = model(mixed_x_ul)

            # Labeled 데이터와 Unlabeled 데이터에 대한 손실 계산
            loss_l = mixup_criterion(criterion, output_l, y_l_a, y_l_b, lam)
            loss_ul = mixup_criterion(criterion, output_ul, y_ul_a, y_ul_b, lam)
            loss = loss_l + loss_ul

            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())

        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val AUC : [{_val_score:.5f}]')

        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model

    return best_model

def multiLabel_AUC(y_true, y_scores):
    auc_scores = []
    for i in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score

def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss, all_labels, all_probs = [], [], []
    with torch.no_grad():
        for features, labels in tqdm(iter(val_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            probs = model(features)
            loss = criterion(probs, labels)
            val_loss.append(loss.item())
            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())
        _val_loss = np.mean(val_loss)
        all_labels = np.concatenate(all_labels, axis=0)
        all_probs = np.concatenate(all_probs, axis=0)
        auc_score = multiLabel_AUC(all_labels, all_probs)
    return _val_loss, auc_score

model = MLP()
optimizer = torch.optim.Adam(params=model.parameters(), lr=CONFIG.LR)
infer_model = train(model, optimizer, train_loader, val_loader, device)

test = pd.read_csv('./test.csv')
test_mfcc = get_mfcc_feature(test, False)
test_dataset = CustomDataset(test_mfcc, None)
test_loader = DataLoader(test_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)

def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for features in tqdm(iter(test_loader)):
            features = features.float().to(device)
            probs = model(features)
            probs = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

preds = inference(infer_model, test_loader, device)
submit = pd.read_csv('./sample_submission.csv')
submit.iloc[:, 1:] = preds
submit.head()
submit.to_csv('./mixmatch_submit.csv', index=False)

44350it [17:56, 41.20it/s]
11088it [04:53, 37.83it/s]
100%|███████████████████████████████████████████████████████████████████████████████| 116/116 [00:00<00:00, 868.80it/s]


Epoch [1], Train Loss : [1.31388] Val Loss : [0.42184] Val AUC : [0.92048]


100%|███████████████████████████████████████████████████████████████████████████████| 116/116 [00:00<00:00, 882.03it/s]


Epoch [2], Train Loss : [1.13205] Val Loss : [0.38781] Val AUC : [0.93421]


100%|███████████████████████████████████████████████████████████████████████████████| 116/116 [00:00<00:00, 690.86it/s]


...

100%|███████████████████████████████████████████████████████████████████████████████| 116/116 [00:00<00:00, 860.50it/s]


Epoch [99], Train Loss : [1.00947] Val Loss : [0.22653] Val AUC : [0.99170]


100%|███████████████████████████████████████████████████████████████████████████████| 116/116 [00:00<00:00, 867.24it/s]


Epoch [100], Train Loss : [0.99390] Val Loss : [0.22736] Val AUC : [0.99251]


50000it [26:46, 31.12it/s]
100%|███████████████████████████████████████████████████████████████████████████████| 521/521 [00:00<00:00, 965.28it/s]


In [5]:
import librosa
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import os
import warnings

warnings.filterwarnings('ignore')

# 설정 클래스
class Config:
    SR = 32000  # 샘플링 레이트
    N_MFCC = 13  # MFCC 계수 개수
    ROOT_FOLDER = './'
    N_CLASSES = 2  # 클래스 개수 (진짜, 가짜)
    N_ESTIMATORS = 50  # AdaBoost에 사용할 약한 학습기의 수
    SEED = 42  # 랜덤 시드

CONFIG = Config()

# 시드 고정 함수
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CONFIG.SEED) # 시드 고정

# 데이터셋 로드
df = pd.read_csv('train.csv')
train, val = train_test_split(df, test_size=0.2, random_state=CONFIG.SEED)

# MFCC 특징 추출 함수
def get_mfcc_feature(df, train_mode=True):
    features = []
    labels = []
    for _, row in tqdm(df.iterrows()):
        y, sr = librosa.load(row['path'], sr=CONFIG.SR)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CONFIG.N_MFCC)
        mfcc = np.mean(mfcc.T, axis=0)
        features.append(mfcc)
        if train_mode:
            labels.append(row['label'])
    return features, labels

train_mfcc, train_labels = get_mfcc_feature(train, True)
val_mfcc, val_labels = get_mfcc_feature(val, True)

# 라벨을 이진화
train_labels = [0 if label == 'fake' else 1 for label in train_labels]
val_labels = [0 if label == 'fake' else 1 for label in val_labels]

# AdaBoost 분류기 초기화 및 학습
# 기본 학습기로 깊이가 1인 결정 트리를 사용
base_estimator = DecisionTreeClassifier(max_depth=1, random_state=CONFIG.SEED)
model = AdaBoostClassifier(estimator=base_estimator, n_estimators=CONFIG.N_ESTIMATORS, random_state=CONFIG.SEED)
model.fit(train_mfcc, train_labels)

# 모델 검증
val_predictions = model.predict_proba(val_mfcc)[:, 1]
val_auc = roc_auc_score(val_labels, val_predictions)
print(f'Validation AUC: {val_auc:.5f}')

# 테스트 데이터셋 로드 및 MFCC 특징 추출
test = pd.read_csv('test.csv')
test_mfcc, _ = get_mfcc_feature(test, False)

# 테스트 데이터 예측
test_predictions = model.predict_proba(test_mfcc)[:, 1]

# 제출 파일 생성
submit = pd.read_csv('sample_submission.csv')
submit.iloc[:, 1] = test_predictions
submit.to_csv('adaboost_submit.csv', index=False)


44350it [09:57, 74.28it/s]
11088it [02:28, 74.87it/s]


Validation AUC: 0.90723


50000it [23:20, 35.71it/s]


In [7]:
!pip install torchmetrics

   ---------------------------------------- 0.0/868.8 kB ? eta -:--:--
   ---------------------------------------- 10.2/868.8 kB ? eta -:--:--
   ---------------------------------------- 10.2/868.8 kB ? eta -:--:--
   ---- ---------------------------------- 92.2/868.8 kB 871.5 kB/s eta 0:00:01
   ---- ---------------------------------- 92.2/868.8 kB 871.5 kB/s eta 0:00:01
   ---------- ----------------------------- 225.3/868.8 kB 1.1 MB/s eta 0:00:01
   ---------- ----------------------------- 225.3/868.8 kB 1.1 MB/s eta 0:00:01
   ------------------------- -------------- 563.2/868.8 kB 1.8 MB/s eta 0:00:01
   --------------------------- ------------ 593.9/868.8 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------  860.2/868.8 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 868.8/868.8 kB 2.0 MB/s eta 0:00:00


In [10]:
import librosa
import numpy as np
import pandas as pd
import random
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
import torchmetrics
import os
import warnings

warnings.filterwarnings('ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class Config:
    SR = 32000  # 샘플링 레이트
    N_MFCC = 13  # MFCC 수
    N_CLASSES = 2  # 클래스 수 (진짜, 가짜)
    BATCH_SIZE = 96  # 배치 크기
    N_EPOCHS = 50  # 에포크 수
    LR = 3e-4  # 학습률
    SEED = 42  # 시드 값

CONFIG = Config()

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CONFIG.SEED) # Seed 고정

# 데이터 불러오기
df = pd.read_csv('train.csv')
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CONFIG.SEED)

# 특징 추출 함수
def get_audio_features(df, train_mode=True):
    features = []
    labels = []
    for _, row in tqdm(df.iterrows()):
        y, sr = librosa.load(row['path'], sr=CONFIG.SR)  # 오디오 파일 로드
        
        # MFCC 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CONFIG.N_MFCC)
        mfcc = np.mean(mfcc.T, axis=0)
        
        # Mel 스펙트로그램 추출
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr)
        mel_spec = np.mean(mel_spec.T, axis=0)
        
        # CQT 추출
        cqt = librosa.cqt(y, sr=sr)
        if cqt.shape[1] == 0:
            continue  # CQT 추출이 실패하면 해당 오디오 샘플을 건너뜁니다.
        cqt = np.mean(cqt.T, axis=0)
        
        # CQCC 추출 (CQT와 DCT를 이용)
        try:
            cqcc = librosa.feature.mfcc(S=librosa.amplitude_to_db(np.abs(cqt.reshape(1, -1))), n_mfcc=CONFIG.N_MFCC)
            cqcc = np.mean(cqcc.T, axis=0)
        except:
            continue  # CQCC 추출이 실패하면 해당 오디오 샘플을 건너뜁니다.
        
        # 특징 벡터 결합
        feature = np.concatenate([mfcc, mel_spec, cqt, cqcc])
        features.append(feature)
        
        if train_mode:
            label = row['label']
            label_vector = np.zeros(CONFIG.N_CLASSES, dtype=float)
            label_vector[0 if label == 'fake' else 1] = 1
            labels.append(label_vector)

    if train_mode:
        return features, labels
    return features

train_features, train_labels = get_audio_features(train, True)
val_features, val_labels = get_audio_features(val, True)


# 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        if self.labels is not None:
            return self.features[index], self.labels[index]
        return self.features[index]

# 데이터 로더 생성
train_dataset = CustomDataset(train_features, train_labels)
val_dataset = CustomDataset(val_features, val_labels)

train_loader = DataLoader(train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)

# CNN 모델 클래스 정의
class CNN(nn.Module):
    def __init__(self, input_dim):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(input_dim, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool = nn.MaxPool1d(2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        return x

# BiLSTM 모델 클래스 정의
class BiLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=2, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

from sklearn.metrics import roc_auc_score

# 모델 훈련 함수
def train(model_cnn, model_lstm, optimizer, train_loader, val_loader, device):
    model_cnn.to(device)
    model_lstm.to(device)
    criterion = nn.BCELoss().to(device)

    best_val_score = 0
    best_model_cnn = None
    best_model_lstm = None

    for epoch in range(1, CONFIG.N_EPOCHS + 1):
        model_cnn.train()
        model_lstm.train()
        train_loss = []
        for features, labels in tqdm(iter(train_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)

            optimizer.zero_grad()

            # CNN 처리
            cnn_out = model_cnn(features.unsqueeze(1))
            cnn_out = cnn_out.permute(0, 2, 1)
            
            # LSTM 처리
            output = model_lstm(cnn_out)
            
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        _val_loss, _val_score = validation(model_cnn, model_lstm, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val AUC : [{_val_score:.5f}]')

        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model_cnn = model_cnn
            best_model_lstm = model_lstm

    return best_model_cnn, best_model_lstm

# AUC 계산 함수
def multiLabel_AUC(y_true, y_scores):
    auc_scores = []
    for i in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score

# 모델 검증 함수
def validation(model_cnn, model_lstm, criterion, val_loader, device):
    model_cnn.eval()
    model_lstm.eval()
    val_loss, all_labels, all_probs = [], [], []

    with torch.no_grad():
        for features, labels in tqdm(iter(val_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)

            # CNN 처리
            cnn_out = model_cnn(features.unsqueeze(1))
            cnn_out = cnn_out.permute(0, 2, 1)

            # LSTM 처리
            probs = model_lstm(cnn_out)

            loss = criterion(probs, labels)

            val_loss.append(loss.item())

            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())

        _val_loss = np.mean(val_loss)

        all_labels = np.concatenate(all_labels, axis=0)
        all_probs = np.concatenate(all_probs, axis=0)

        auc_score = multiLabel_AUC(all_labels, all_probs)

    return _val_loss, auc_score

# CNN과 BiLSTM 모델 초기화
model_cnn = CNN(input_dim=1)
model_lstm = BiLSTM(input_dim=128, hidden_dim=128, output_dim=CONFIG.N_CLASSES)
optimizer = torch.optim.Adam(list(model_cnn.parameters()) + list(model_lstm.parameters()), lr=CONFIG.LR)

# 모델 훈련
best_model_cnn, best_model_lstm = train(model_cnn, model_lstm, optimizer, train_loader, val_loader, device)

# 테스트 데이터 로드 및 특징 추출
test = pd.read_csv('./test.csv')
test_features = get_audio_features(test, False)
test_dataset = CustomDataset(test_features, None)
test_loader = DataLoader(test_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)

# 모델 추론 함수
def inference(model_cnn, model_lstm, test_loader, device):
    model_cnn.to(device)
    model_lstm.to(device)
    model_cnn.eval()
    model_lstm.eval()
    predictions = []
    with torch.no_grad():
        for features in tqdm(iter(test_loader)):
            features = features.float().to(device)

            # CNN 처리
            cnn_out = model_cnn(features.unsqueeze(1))
            cnn_out = cnn_out.permute(0, 2, 1)

            # LSTM 처리
            probs = model_lstm(cnn_out)

            probs = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

# 추론 수행
preds = inference(best_model_cnn, best_model_lstm, test_loader, device)

# 결과 저장
submit = pd.read_csv('./sample_submission.csv')
submit.iloc[:, 1:] = preds
submit.head()

submit.to_csv('./baseline_submit.csv', index=False)


44350it [58:12, 12.70it/s]
11088it [13:52, 13.32it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 116/116 [00:06<00:00, 17.33it/s]


Epoch [1], Train Loss : [0.51195] Val Loss : [0.44214] Val AUC : [0.87698]


100%|████████████████████████████████████████████████████████████████████████████████| 116/116 [00:06<00:00, 16.61it/s]


Epoch [2], Train Loss : [0.29287] Val Loss : [0.13380] Val AUC : [0.99013]


100%|████████████████████████████████████████████████████████████████████████████████| 116/116 [00:06<00:00, 18.97it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████| 116/116 [00:05<00:00, 19.95it/s]


Epoch [49], Train Loss : [0.00778] Val Loss : [0.02605] Val AUC : [0.99944]


100%|████████████████████████████████████████████████████████████████████████████████| 116/116 [00:05<00:00, 19.67it/s]


Epoch [50], Train Loss : [0.00901] Val Loss : [0.01955] Val AUC : [0.99974]


50000it [50:48, 16.40it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 521/521 [00:24<00:00, 20.98it/s]
